# Parsing Modules
Modules used to parse the Link Labs GPS Tracker payload.

In [71]:
'''--------------------------------------------------------------------------'''
'''
DESCRIPTION
    This module parses and decodes the payload delivered by the Link Labs
    GPS Tracker and places it in a Python dictionary object with the
    following form:

    { 'PayL': '10174D9BEBD1C13F1B0021FFE5', 'Msg Cnt': 4, 'Msg Type': 'GPS',
      'Lat': 39.0962155, 'Lon': -77.5864549, 'Alt': 33, 'Batt': 4.23,
      'Reserved': 'N/A' }

REFERENCE MATERIALS
    * https://stackoverflow.com/questions/6727875/hex-string-to-signed-int-in-python-3-2           # noqa
    * https://www.binaryhexconverter.com/hex-to-binary-converter
    * http://www.binaryconvert.com/convert_signed_int.html

CREATED BY
        Jeff Irland (jeffrey.irland@verizon.com) in April 2018
'''

# mapping of hex characters to binary repensetation in ascii
hex2bin_map = {'0': '0000', '1': '0001', '2': '0010', '3': '0011', '4': '0100',
               '5': '0101', '6': '0110', '7': '0111', '8': '1000', '9': '1001',
               'A': '1010', 'B': '1011', 'C': '1100', 'D': '1101', 'E': '1110',
               'F': '1111', 'a': '1010', 'b': '1011', 'c': '1100', 'd': '1101',
               'e': '1110', 'f': '1111'}


def HextoBin(hexstring):
    '''Convert a hex encoded ascii string to a binary encoded ascii string.
    '''
    binarystring = ''.join(hex2bin_map[i] for i in hexstring)
    return binarystring


def BintoInt(binarystring):
    '''Convert binary encoded ascii string to integer data.
    '''
    return int(binarystring, 2)


def HextoInt(hexstring):
    '''Convert a hex encoded ascii string to integer data.
    '''
    return BintoInt(HextoBin(hexstring))


def HextoDec(hexstring):
    '''Convert a hex encoded ascii string to signed decimal data. This assumes
    that source is the proper length, and the sign bit is the first bit in the
    first byte of the correct length.  For example:
    HextoDec('F') = -1 and HextoDec('0F') = 15
    '''
    if not isinstance(hexstring, str):
        raise ValueError('string type required')
    if len(hexstring) == 0:
        raise ValueError('string is empty')

    sign_bit_mask = 1 << (len(hexstring) * 4 - 1)
    other_bits_mask = sign_bit_mask - 1
    value = int(hexstring, 16)

    return -(value & sign_bit_mask) | (value & other_bits_mask)


def PayloadParser(payload):
    '''Take a single payload and parse it into its discrete compoents
    but still hex encoded.  These compoents, when decoded, will become
    the message count, message type, latitude, longitude, altitude,
    battery voltage, and a reserved string returned as a dictionary object.
    '''
    payloadparsed = {'PayL': payload, 'Byte_0': payload[0:2],
                     'Byte_1-4': payload[2:10], 'Byte_5-8': payload[10:18],
                     'Byte_9-10': payload[18:22], 'Byte_11-12': payload[22:]}

    return payloadparsed


def PayloadDecoder(payload):
    '''Take a single payload and parse it into a dictionary object with
    message count, message type, latitude, longitude, altitude,
    and battery voltage.
    '''
    payloadparsed = PayloadParser(payload)

    # this is the orginal payload string encoded and unparsed
    payloaddecoded = {'PayL': payloadparsed['PayL']}

    # first 6 bits of the hex formated single byte string gives you the message count              # noqa
    x = HextoBin(payloadparsed['Byte_0'])
    x = x[:6]
    payloaddecoded.update({'Msg Cnt': BintoInt(x)})

    # last 2 bits of the hex formated single byte string gives you the message type                # noqa
    x = HextoBin(payloadparsed['Byte_0'])
    if x[6:] == '00':
        x = 'GPS'
    else:
        x = 'Unknown'
    payloaddecoded.update({'Msg Type': x})

    # from the hex formated 4 byte string, convert it to a signed decimal number
    # For Lat/Long, convert from hex to decimal and multiply by 1.0e-7.
    # These numbers are signed so be careful during the conversion.
    x = HextoDec(payloadparsed['Byte_1-4']) * 1.0E-7
    payloaddecoded.update({'Lat': round(x, 7)})

    # from the hex formated 4 byte string, convert it to a signed decimal number
    # For Lat/Long, convert from hex to decimal and multiply by 1.0e-7.
    # These numbers are signed so be careful during the conversion.
    x = HextoDec(payloadparsed['Byte_5-8']) * 1.0E-7
    payloaddecoded.update({'Lon': round(x, 7)})

    # from the hex formated 2 byte string, convert it to a signed decimal number
    payloaddecoded.update({'Alt': HextoDec(payloadparsed['Byte_9-10'])})

    # The first 10 bits of this hex formated 2 byte string is a ADC reading
    # for the battery voltage.  Convert the ADC reading to unsigned intiger
    # use this formula: ( 13.1 * ADC ) / (3.1 * 1023)
    # maximum value will be  4.22V
    x = HextoBin(payloadparsed['Byte_11-12'])
    x = x[:10]
    x = BintoInt(x) * 13.1 / (3.1 * 1023)
    payloaddecoded.update({'Batt': round(x, 2)})

    # the last 6 bits of this hex formated 2 byte string isn't currently used
    x = HextoBin(payloadparsed['Byte_11-12'])
    x = x[10:]
    payloaddecoded.update({'Reserved': 'N/A'})

    return payloaddecoded

# Unit Tests
Unit tests for the Link Labs GPS Tracker payload parser module.

In [72]:
'''--------------------------------------------------------------------------'''
'''
DESCRIPTION
    This module provides unit test routines for the parsing module.

REFERENCE MATERIALS
    pytest framework - https://docs.pytest.org/

CREATED BY
        Jeff Irland (jeffrey.irland@verizon.com) in April 2018
'''


# import the necessary packages
import pytest


# test cases for HextoBin, HextoInt, and BintoInt
CASE1 = [{'hex': '10', 'bin': '00010000', 'int': 16}]
CASE1.append({'hex': '4F5', 'bin': '010011110101', 'int': 1269})
CASE1.append({'hex': 'A37F', 'bin': '1010001101111111', 'int': 41855})
CASE1.append({'hex': 'c3a4c3b6c3bc',
              'bin': '110000111010010011000011101101101100001110111100',
              'int': 215112425587644})
CASE1.append({'hex': '3249CD52F37FF57D',
              'bin': '0011001001001001110011010101001011110011011111111111010101111101',           # noqa
              'int': 3623653131352536445})

# test cases for PayloadParser and PayloadDecoder
CASE2 = [{'pl': '10174D9BEBD1C13F1B0021FFE5',
          'plp': {'PayL': '10174D9BEBD1C13F1B0021FFE5', 'Byte_0': '10',
                  'Byte_1-4': '174D9BEB', 'Byte_5-8': 'D1C13F1B',
                  'Byte_9-10': '0021', 'Byte_11-12': 'FFE5'},
          'pld': {'PayL': '10174D9BEBD1C13F1B0021FFE5', 'Msg Cnt': 4,
                  'Msg Type': 'GPS', 'Lat': 39.0962155,
                  'Lon': -77.5864549, 'Alt': 33, 'Batt': 4.23,
                  'Reserved': 'N/A'}}]
CASE2.append({'pl': '04174D918ED1C13B40007AFFE5',
              'plp': {'PayL': '04174D918ED1C13B40007AFFE5', 'Byte_0': '04',
                      'Byte_1-4': '174D918E', 'Byte_5-8': 'D1C13B40',
                      'Byte_9-10': '007A', 'Byte_11-12': 'FFE5'},
              'pld': {'PayL': '04174D918ED1C13B40007AFFE5', 'Msg Cnt': 1,
                      'Msg Type': 'GPS', 'Lat': 39.0959502, 'Lon': -77.5865536,
                      'Alt': 122, 'Batt': 4.23, 'Reserved': 'N/A'}})


# execute all the unit tests below
def test_unit():
    test_HextoBin()
    test_BintoInt()
    test_HextoInt()
    test_PayloadParser()
    test_PayloadDecoder()


def test_HextoBin():
    for i in range(len(CASE1)):
        value = HextoBin(CASE1[i]['hex'])
        assert value == CASE1[i]['bin']


def test_BintoInt():
    for i in range(len(CASE1)):
        value = BintoInt(CASE1[i]['bin'])
        assert value == CASE1[i]['int']


def test_HextoInt():
    for i in range(len(CASE1)):
        value = HextoInt(CASE1[i]['hex'])
        assert value == CASE1[i]['int']


def test_PayloadParser():
    for i in range(len(CASE2)):
        value = PayloadParser(CASE2[i]['pl'])
        assert value == CASE2[i]['plp']


def test_PayloadDecoder():
    for i in range(len(CASE2)):
        value = PayloadDecoder(CASE2[i]['pl'])
        assert value == CASE2[i]['pld']

# Commandline Script
Commandline script for parsing the Link Labs GPS payload.

In [73]:
'''--------------------------------------------------------------------------'''
'''
DESCRIPTION
    This script decodes the payload delivered by the Link Labs GPS Tracker.

USAGE
    python3 tkrdecoder.py [--format table | json ] payload [ payload ... ]

REFERENCE MATERIALS
    * https://pymotw.com/3/argparse/

CREATED BY
        Jeff Irland (jeffrey.irland@verizon.com) in April 2018
'''


# import the necessary packages
import sys
import json
import argparse


def LineArgumentParser():
    '''Construct the commandline argument parser, add the rules for the
    arguments, and then parse the arguments (found in sys.argv).
    '''
    list = ['table', 'json']        # output format options

    ap = argparse.ArgumentParser(
        prog='tkrdecoder',
        description='This script parses the payload delivered by the \
        Link Labs GPS Tracker.',
        epilog='See XXX for additional information.')

    ap.add_argument('-f', '--format',
                    required=False,
                    choices=list,
                    default='table',
                    help='format of the output with allowed values of \'' +
                    '\', \''.join(list) + '\'.',
                    metavar='')

    ap.add_argument('payload',
                    nargs='+',
                    help='payload(s) from the Link Labs Cat-M1 GPS Tracker')

    ap.add_argument('--version', action='version',
                    version='%(prog)s 0.1')

    return vars(ap.parse_args())


def PrintHeader():
    print('\t\t' + '\t\tMessage' + '\tMessage' + '\t' + '\t' +
          '\t' + '\t\t\t  Battery' + '\t')
    print('Payload' + '\t\t\t\t Count' + '\t Type' + '\tLatitude' +
          '\tLongitude' + '\tAltitude' + '  Voltage' + '\tReserved')


def PrintTable(parsedpayload):
    print(parsedpayload['PayL'], '\t   ', parsedpayload['Msg Cnt'], '\t  ',
          parsedpayload['Msg Type'], '\t', parsedpayload['Lat'], '\t',
          parsedpayload['Lon'], '\t  ', parsedpayload['Alt'], '\t   ',
          parsedpayload['Batt'], '\t\t  ', parsedpayload['Reserved'], sep='')


if __name__ == '__main__':
    # manually creating the command line since you are within Jupyter
    sys.argv = ['tkrdecoder.py', '-f', 'table',
                '10174D9BEBD1C13F1B0021FFE5', '04174D918ED1C13B40007AFFE5']

    # parse the commandline arguments
    args = LineArgumentParser()

    # if doing a table output, print the table headers
    if args['format'] == 'table':
        PrintHeader()

    # decode payloads from the commandline
    for pl in args['payload']:
        decoded_payload = PayloadDecoder(pl)

        # print a formated output of the payload strings
        if args['format'] == 'table':
            PrintTable(decoded_payload)
        else:
            print(json.dumps(decoded_payload))

    # unit testing
    test_unit()

				Message	Message						  Battery	
Payload				 Count	 Type	Latitude	Longitude	Altitude  Voltage	Reserved
10174D9BEBD1C13F1B0021FFE5	   4	  GPS	39.0962155	-77.5864549	  33	   4.23		  N/A
04174D918ED1C13B40007AFFE5	   1	  GPS	39.0959502	-77.5865536	  122	   4.23		  N/A
